# Chapter 3: ギブスサンプリング

## 学習目標
- ギブスサンプリングの基本原理を理解する
- 条件付き分布の導出方法を学ぶ
- 多変量正規分布での実装を習得する
- 混合モデルや階層モデルへの応用を理解する
- ブロックギブスサンプリングの概念を学ぶ

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.special import logsumexp
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")
np.random.seed(42)

## 3.1 ギブスサンプリングの基本原理

ギブスサンプリングは、多変量分布 $p(x_1, x_2, ..., x_k)$ から各変数を条件付き分布に基づいて順番に更新する手法です。

### アルゴリズム（k変数の場合）
1. 初期値 $(x_1^{(0)}, x_2^{(0)}, ..., x_k^{(0)})$ を設定
2. 各イテレーション $t$ で以下を実行：
   - $x_1^{(t+1)} \sim p(x_1 | x_2^{(t)}, x_3^{(t)}, ..., x_k^{(t)})$
   - $x_2^{(t+1)} \sim p(x_2 | x_1^{(t+1)}, x_3^{(t)}, ..., x_k^{(t)})$
   - $\vdots$
   - $x_k^{(t+1)} \sim p(x_k | x_1^{(t+1)}, x_2^{(t+1)}, ..., x_{k-1}^{(t+1)})$

### 重要な性質
- **受理率100%**：条件付き分布から直接サンプリング
- **自動的な詳細釣り合い**：条件付き分布の性質により満たされる
- **計算効率**：複雑な受理確率計算が不要

## 3.2 例1：2変量正規分布からのサンプリング

まず、解析的に条件付き分布を導出できる2変量正規分布でギブスサンプリングを実装してみましょう。

In [ ]:
def gibbs_sampling_bivariate_normal(mu, cov, n_samples, initial_value=None):
    """
    2変量正規分布からのギブスサンプリング
    
    Parameters:
    - mu: 平均ベクトル [mu_x, mu_y]
    - cov: 共分散行列 [[var_x, cov_xy], [cov_xy, var_y]]
    - n_samples: サンプル数
    - initial_value: 初期値 [x0, y0]
    
    Returns:
    - samples: shape (n_samples, 2) のサンプル配列
    """
    samples = np.zeros((n_samples, 2))
    
    # 条件付き分布のパラメータを事前計算
    mu_x, mu_y = mu[0], mu[1]
    var_x, var_y = cov[0, 0], cov[1, 1]
    cov_xy = cov[0, 1]
    
    # 相関係数
    rho = cov_xy / np.sqrt(var_x * var_y)
    
    # 条件付き分布の標準偏差
    sigma_x_given_y = np.sqrt(var_x * (1 - rho**2))
    sigma_y_given_x = np.sqrt(var_y * (1 - rho**2))
    
    # 初期値の設定
    if initial_value is None:
        x, y = mu_x, mu_y
    else:
        x, y = initial_value[0], initial_value[1]
    
    for i in range(n_samples):
        # x | y からサンプリング
        # p(x|y) ~ N(mu_x + rho*(sigma_x/sigma_y)*(y - mu_y), sigma_x^2*(1-rho^2))
        mu_x_given_y = mu_x + rho * np.sqrt(var_x / var_y) * (y - mu_y)
        x = np.random.normal(mu_x_given_y, sigma_x_given_y)
        
        # y | x からサンプリング
        # p(y|x) ~ N(mu_y + rho*(sigma_y/sigma_x)*(x - mu_x), sigma_y^2*(1-rho^2))
        mu_y_given_x = mu_y + rho * np.sqrt(var_y / var_x) * (x - mu_x)
        y = np.random.normal(mu_y_given_x, sigma_y_given_x)
        
        samples[i] = [x, y]
    
    return samples

# パラメータ設定
mu = np.array([1.0, 2.0])
cov = np.array([[2.0, 1.5], [1.5, 3.0]])

print(f"目標分布の平均: {mu}")
print(f"目標分布の共分散:\n{cov}")
print(f"相関係数: {cov[0,1]/np.sqrt(cov[0,0]*cov[1,1]):.3f}")

# ギブスサンプリング実行
samples_gibbs = gibbs_sampling_bivariate_normal(mu, cov, 10000)

# 結果の統計
burnin = 1000
sample_mean = np.mean(samples_gibbs[burnin:], axis=0)
sample_cov = np.cov(samples_gibbs[burnin:].T)

print(f"\nサンプル平均: {sample_mean}")
print(f"サンプル共分散:\n{sample_cov}")
print(f"サンプル相関係数: {sample_cov[0,1]/np.sqrt(sample_cov[0,0]*sample_cov[1,1]):.3f}")

In [ ]:
# ギブスサンプリング結果の可視化
def plot_gibbs_results(samples, mu, cov, title="Gibbs Sampling Results"):
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    burnin = len(samples) // 10
    samples_clean = samples[burnin:]
    
    # サンプルの軌跡（最初の500サンプル）
    trajectory = samples[:500]
    axes[0, 0].plot(trajectory[:, 0], trajectory[:, 1], 'b-', alpha=0.7, linewidth=0.8)
    axes[0, 0].plot(trajectory[:, 0], trajectory[:, 1], 'b.', markersize=2, alpha=0.8)
    axes[0, 0].plot(trajectory[0, 0], trajectory[0, 1], 'go', markersize=8, label='Start')
    axes[0, 0].plot(trajectory[-1, 0], trajectory[-1, 1], 'ro', markersize=8, label='End')
    axes[0, 0].set_title('Gibbs Sampling Trajectory (first 500)')
    axes[0, 0].set_xlabel('X1')
    axes[0, 0].set_ylabel('X2')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 散布図と真の分布の等高線
    axes[0, 1].scatter(samples_clean[::5, 0], samples_clean[::5, 1], alpha=0.6, s=1)
    
    # 真の分布の等高線
    x1_range = np.linspace(samples_clean[:, 0].min(), samples_clean[:, 0].max(), 50)
    x2_range = np.linspace(samples_clean[:, 1].min(), samples_clean[:, 1].max(), 50)
    X1, X2 = np.meshgrid(x1_range, x2_range)
    pos = np.dstack((X1, X2))
    rv = stats.multivariate_normal(mu, cov)
    axes[0, 1].contour(X1, X2, rv.pdf(pos), colors='red', alpha=0.8, linewidths=2)
    axes[0, 1].set_title('Samples with True Distribution')
    axes[0, 1].set_xlabel('X1')
    axes[0, 1].set_ylabel('X2')
    axes[0, 1].set_aspect('equal')
    
    # トレースプロット
    axes[0, 2].plot(samples[:2000, 0], alpha=0.7, label='X1', linewidth=0.8)
    axes[0, 2].plot(samples[:2000, 1], alpha=0.7, label='X2', linewidth=0.8)
    axes[0, 2].axvline(burnin, color='red', linestyle='--', alpha=0.7, label='Burn-in')
    axes[0, 2].set_title('Trace Plot')
    axes[0, 2].set_xlabel('Iteration')
    axes[0, 2].set_ylabel('Value')
    axes[0, 2].legend()
    
    # X1のマージナル分布
    axes[1, 0].hist(samples_clean[:, 0], bins=50, density=True, alpha=0.7, 
                    color='lightblue', label='X1 samples')
    x1_theory = np.linspace(samples_clean[:, 0].min(), samples_clean[:, 0].max(), 100)
    axes[1, 0].plot(x1_theory, stats.norm.pdf(x1_theory, mu[0], np.sqrt(cov[0, 0])), 
                    'r-', linewidth=2, label='X1 true')
    axes[1, 0].set_title('Marginal Distribution X1')
    axes[1, 0].set_xlabel('X1')
    axes[1, 0].set_ylabel('Density')
    axes[1, 0].legend()
    
    # X2のマージナル分布
    axes[1, 1].hist(samples_clean[:, 1], bins=50, density=True, alpha=0.7, 
                    color='lightgreen', label='X2 samples')
    x2_theory = np.linspace(samples_clean[:, 1].min(), samples_clean[:, 1].max(), 100)
    axes[1, 1].plot(x2_theory, stats.norm.pdf(x2_theory, mu[1], np.sqrt(cov[1, 1])), 
                    'r-', linewidth=2, label='X2 true')
    axes[1, 1].set_title('Marginal Distribution X2')
    axes[1, 1].set_xlabel('X2')
    axes[1, 1].set_ylabel('Density')
    axes[1, 1].legend()
    
    # 自己相関（X1について）
    from statsmodels.tsa.stattools import acf
    lags = min(100, len(samples_clean) // 10)
    autocorr_x1 = acf(samples_clean[:, 0], nlags=lags, fft=True)
    autocorr_x2 = acf(samples_clean[:, 1], nlags=lags, fft=True)
    
    axes[1, 2].plot(autocorr_x1, label='X1', alpha=0.8)
    axes[1, 2].plot(autocorr_x2, label='X2', alpha=0.8)
    axes[1, 2].axhline(0, color='k', linestyle='--', alpha=0.5)
    axes[1, 2].axhline(0.05, color='r', linestyle='--', alpha=0.5)
    axes[1, 2].axhline(-0.05, color='r', linestyle='--', alpha=0.5)
    axes[1, 2].set_title('Autocorrelation Functions')
    axes[1, 2].set_xlabel('Lag')
    axes[1, 2].set_ylabel('ACF')
    axes[1, 2].legend()
    
    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

plot_gibbs_results(samples_gibbs, mu, cov)

## 3.3 メトロポリス・ヘイスティングス法との比較

同じ分布に対してメトロポリス・ヘイスティングス法も適用し、性能を比較してみましょう。

In [ ]:
# メトロポリス・ヘイスティングス法の実装（前章から）
def multivariate_normal_log_pdf(x, mu, cov):
    """多変量正規分布の対数確率密度"""
    k = len(mu)
    diff = x - mu
    
    try:
        chol = np.linalg.cholesky(cov)
        log_det = 2 * np.sum(np.log(np.diag(chol)))
        solve = np.linalg.solve(chol, diff)
        mahalanobis_sq = np.sum(solve**2)
    except np.linalg.LinAlgError:
        return -np.inf
    
    return -0.5 * (k * np.log(2 * np.pi) + log_det + mahalanobis_sq)

def mh_multivariate_normal(mu, cov, n_samples, step_size=0.5):
    """多変量正規分布からのMHサンプリング"""
    samples = np.zeros((n_samples, len(mu)))
    current = np.copy(mu)  # 平均から開始
    current_log_prob = multivariate_normal_log_pdf(current, mu, cov)
    n_accepted = 0
    
    cov_proposal = step_size * np.eye(len(mu))
    
    for i in range(n_samples):
        # 提案
        proposed = np.random.multivariate_normal(current, cov_proposal)
        proposed_log_prob = multivariate_normal_log_pdf(proposed, mu, cov)
        
        # 受理確率（対称提案なので簡単）
        log_alpha = proposed_log_prob - current_log_prob
        alpha = min(1.0, np.exp(log_alpha))
        
        # 受理/棄却
        if np.random.rand() < alpha:
            current = proposed
            current_log_prob = proposed_log_prob
            n_accepted += 1
        
        samples[i] = current
    
    return samples, n_accepted / n_samples

# MHサンプリング実行
print("メトロポリス・ヘイスティングス法でサンプリング中...")
samples_mh, acceptance_rate = mh_multivariate_normal(mu, cov, 10000, step_size=0.8)

print(f"MH法受理率: {acceptance_rate:.3f}")

# 統計の比較
burnin = 1000

# ギブス統計
gibbs_mean = np.mean(samples_gibbs[burnin:], axis=0)
gibbs_cov = np.cov(samples_gibbs[burnin:].T)

# MH統計
mh_mean = np.mean(samples_mh[burnin:], axis=0)
mh_cov = np.cov(samples_mh[burnin:].T)

print(f"\n=== 統計比較 ===")
print(f"真の平均:     {mu}")
print(f"ギブス平均:   {gibbs_mean}")
print(f"MH平均:       {mh_mean}")
print(f"\n真の共分散:")
print(cov)
print(f"ギブス共分散:")
print(gibbs_cov)
print(f"MH共分散:")
print(mh_cov)


In [ ]:
# 効率の比較
from statsmodels.tsa.stattools import acf

def compute_efficiency_metrics(samples, burnin_frac=0.1):
    """効率指標の計算"""
    burnin = int(len(samples) * burnin_frac)
    clean_samples = samples[burnin:]
    
    # 各次元の自己相関時間を計算
    autocorr_times = []
    eff_sample_sizes = []
    
    for dim in range(clean_samples.shape[1]):
        data = clean_samples[:, dim]
        lags = min(200, len(data) // 4)
        autocorr = acf(data, nlags=lags, fft=True)
        
        # 最初に閾値を下回るラグを見つける
        tau_int = 1
        for lag in range(1, len(autocorr)):
            if autocorr[lag] < 0.05:
                tau_int = lag
                break
        
        autocorr_times.append(tau_int)
        eff_sample_sizes.append(len(data) / (2 * tau_int + 1))
    
    return autocorr_times, eff_sample_sizes

# 効率比較
gibbs_autocorr, gibbs_eff = compute_efficiency_metrics(samples_gibbs)
mh_autocorr, mh_eff = compute_efficiency_metrics(samples_mh)

print("=== 効率比較 ===")
print(f"{'Method':<10} {'Dim':<5} {'Autocorr Time':<15} {'Eff Sample Size':<18}")
print("-" * 50)
for dim in range(2):
    print(f"{'Gibbs':<10} {'X'+str(dim+1):<5} {gibbs_autocorr[dim]:<15d} {gibbs_eff[dim]:<18.1f}")
    print(f"{'MH':<10} {'X'+str(dim+1):<5} {mh_autocorr[dim]:<15d} {mh_eff[dim]:<18.1f}")
    print()

print(f"平均効率サンプルサイズ:")
print(f"  ギブス: {np.mean(gibbs_eff):.1f}")
print(f"  MH:     {np.mean(mh_eff):.1f}")

## 3.4 例2：混合正規分布の推定

より実践的な例として、潜在変数を持つ混合正規分布のパラメータ推定をギブスサンプリングで行ってみましょう。

In [ ]:
# データ生成
def generate_mixture_data(n_samples, weights, means, variances):
    """
    混合正規分布からデータを生成
    
    Parameters:
    - n_samples: サンプル数
    - weights: 混合重み
    - means: 各成分の平均
    - variances: 各成分の分散
    """
    n_components = len(weights)
    
    # 成分の割り当て
    components = np.random.choice(n_components, size=n_samples, p=weights)
    
    # データ生成
    data = np.zeros(n_samples)
    for i in range(n_samples):
        comp = components[i]
        data[i] = np.random.normal(means[comp], np.sqrt(variances[comp]))
    
    return data, components

# 真のパラメータ
true_weights = np.array([0.3, 0.7])
true_means = np.array([-2.0, 2.0])
true_variances = np.array([0.5, 1.0])

# データ生成
n_obs = 200
data, true_components = generate_mixture_data(n_obs, true_weights, true_means, true_variances)

print(f"真のパラメータ:")
print(f"  重み: {true_weights}")
print(f"  平均: {true_means}")
print(f"  分散: {true_variances}")

# データの可視化
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.hist(data, bins=30, density=True, alpha=0.7, color='lightblue')
x_range = np.linspace(data.min(), data.max(), 1000)
true_density = (true_weights[0] * stats.norm.pdf(x_range, true_means[0], np.sqrt(true_variances[0])) +
                true_weights[1] * stats.norm.pdf(x_range, true_means[1], np.sqrt(true_variances[1])))
plt.plot(x_range, true_density, 'r-', linewidth=2, label='True distribution')
plt.title('Generated Data')
plt.xlabel('Value')
plt.ylabel('Density')
plt.legend()

plt.subplot(1, 2, 2)
colors = ['red', 'blue']
for k in range(2):
    mask = true_components == k
    plt.hist(data[mask], bins=15, alpha=0.7, color=colors[k], 
             label=f'Component {k+1}', density=True)
plt.title('Data by True Components')
plt.xlabel('Value')
plt.ylabel('Density')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
def gibbs_mixture_gaussian(data, n_components, n_iterations, 
                          prior_alpha=1.0, prior_mu_var=10.0, 
                          prior_sigma_shape=1.0, prior_sigma_scale=1.0):
    """
    混合正規分布のギブスサンプリング
    
    Parameters:
    - data: 観測データ
    - n_components: 混合成分数
    - n_iterations: イテレーション数
    - prior_*: 事前分布のパラメータ
    """
    n_obs = len(data)
    
    # パラメータ保存用配列
    weights_samples = np.zeros((n_iterations, n_components))
    means_samples = np.zeros((n_iterations, n_components))
    variances_samples = np.zeros((n_iterations, n_components))
    components_samples = np.zeros((n_iterations, n_obs), dtype=int)
    
    # 初期値
    weights = np.ones(n_components) / n_components
    means = np.random.normal(0, 2, n_components)
    variances = np.ones(n_components)
    components = np.random.choice(n_components, n_obs)
    
    for iteration in range(n_iterations):
        # 1. 潜在変数（成分割り当て）の更新
        for i in range(n_obs):
            # 各成分への所属確率を計算
            log_probs = np.zeros(n_components)
            for k in range(n_components):
                log_probs[k] = (np.log(weights[k]) + 
                               stats.norm.logpdf(data[i], means[k], np.sqrt(variances[k])))
            
            # 安定な確率計算
            log_probs -= logsumexp(log_probs)
            probs = np.exp(log_probs)
            
            # サンプリング
            components[i] = np.random.choice(n_components, p=probs)
        
        # 2. 混合重みの更新（ディリクレ分布から）
        counts = np.bincount(components, minlength=n_components)
        weights = np.random.dirichlet(prior_alpha + counts)
        
        # 3. 平均の更新（正規分布から）
        for k in range(n_components):
            mask = components == k
            n_k = np.sum(mask)
            
            if n_k > 0:
                data_k = data[mask]
                sample_mean = np.mean(data_k)
                
                # 事後分布のパラメータ
                posterior_var = 1 / (1/prior_mu_var + n_k/variances[k])
                posterior_mean = posterior_var * (n_k * sample_mean / variances[k])
                
                means[k] = np.random.normal(posterior_mean, np.sqrt(posterior_var))
            else:
                # データが割り当てられていない場合は事前分布から
                means[k] = np.random.normal(0, np.sqrt(prior_mu_var))
        
        # 4. 分散の更新（逆ガンマ分布から）
        for k in range(n_components):
            mask = components == k
            n_k = np.sum(mask)
            
            if n_k > 0:
                data_k = data[mask]
                ss = np.sum((data_k - means[k])**2)
                
                # 事後分布のパラメータ
                posterior_shape = prior_sigma_shape + n_k / 2
                posterior_scale = prior_sigma_scale + ss / 2
                
                # 逆ガンマからサンプリング（ガンマの逆数）
                variances[k] = 1 / np.random.gamma(posterior_shape, 1/posterior_scale)
            else:
                # データが割り当てられていない場合は事前分布から
                variances[k] = 1 / np.random.gamma(prior_sigma_shape, 1/prior_sigma_scale)
        
        # サンプル保存
        weights_samples[iteration] = weights
        means_samples[iteration] = means
        variances_samples[iteration] = variances
        components_samples[iteration] = components
        
        if iteration % 200 == 0:
            print(f"Iteration {iteration}: weights={weights:.3f}, "
                  f"means=[{means[0]:.2f}, {means[1]:.2f}], "
                  f"vars=[{variances[0]:.2f}, {variances[1]:.2f}]")
    
    return {
        'weights': weights_samples,
        'means': means_samples,
        'variances': variances_samples,
        'components': components_samples
    }

# ギブスサンプリング実行
print("混合正規分布のパラメータ推定中...")
results = gibbs_mixture_gaussian(data, n_components=2, n_iterations=2000)

# 結果の統計
burnin = 500
weights_post = results['weights'][burnin:]
means_post = results['means'][burnin:]
variances_post = results['variances'][burnin:]

print(f"\n=== 推定結果 ===")
print(f"重み:")
print(f"  真値:   {true_weights}")
print(f"  推定値: {np.mean(weights_post, axis=0)}")
print(f"平均:")
print(f"  真値:   {true_means}")
print(f"  推定値: {np.mean(means_post, axis=0)}")
print(f"分散:")
print(f"  真値:   {true_variances}")
print(f"  推定値: {np.mean(variances_post, axis=0)}")

In [ ]:
# 混合モデル結果の可視化
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# パラメータのトレースプロット
iterations = np.arange(len(results['weights']))

# 重み
axes[0, 0].plot(iterations, results['weights'][:, 0], alpha=0.8, label='Component 1')
axes[0, 0].plot(iterations, results['weights'][:, 1], alpha=0.8, label='Component 2')
axes[0, 0].axhline(true_weights[0], color='red', linestyle='--', alpha=0.7)
axes[0, 0].axhline(true_weights[1], color='blue', linestyle='--', alpha=0.7)
axes[0, 0].axvline(burnin, color='gray', linestyle=':', alpha=0.7)
axes[0, 0].set_title('Mixture Weights')
axes[0, 0].set_xlabel('Iteration')
axes[0, 0].set_ylabel('Weight')
axes[0, 0].legend()

# 平均
axes[0, 1].plot(iterations, results['means'][:, 0], alpha=0.8, label='Component 1')
axes[0, 1].plot(iterations, results['means'][:, 1], alpha=0.8, label='Component 2')
axes[0, 1].axhline(true_means[0], color='red', linestyle='--', alpha=0.7)
axes[0, 1].axhline(true_means[1], color='blue', linestyle='--', alpha=0.7)
axes[0, 1].axvline(burnin, color='gray', linestyle=':', alpha=0.7)
axes[0, 1].set_title('Component Means')
axes[0, 1].set_xlabel('Iteration')
axes[0, 1].set_ylabel('Mean')
axes[0, 1].legend()

# 分散
axes[0, 2].plot(iterations, results['variances'][:, 0], alpha=0.8, label='Component 1')
axes[0, 2].plot(iterations, results['variances'][:, 1], alpha=0.8, label='Component 2')
axes[0, 2].axhline(true_variances[0], color='red', linestyle='--', alpha=0.7)
axes[0, 2].axhline(true_variances[1], color='blue', linestyle='--', alpha=0.7)
axes[0, 2].axvline(burnin, color='gray', linestyle=':', alpha=0.7)
axes[0, 2].set_title('Component Variances')
axes[0, 2].set_xlabel('Iteration')
axes[0, 2].set_ylabel('Variance')
axes[0, 2].legend()

# 事後分布のヒストグラム
axes[1, 0].hist(weights_post[:, 0], bins=30, alpha=0.7, density=True, label='Component 1')
axes[1, 0].hist(weights_post[:, 1], bins=30, alpha=0.7, density=True, label='Component 2')
axes[1, 0].axvline(true_weights[0], color='red', linestyle='--', label='True values')
axes[1, 0].axvline(true_weights[1], color='blue', linestyle='--')
axes[1, 0].set_title('Posterior Distribution of Weights')
axes[1, 0].set_xlabel('Weight')
axes[1, 0].set_ylabel('Density')
axes[1, 0].legend()

axes[1, 1].hist(means_post[:, 0], bins=30, alpha=0.7, density=True, label='Component 1')
axes[1, 1].hist(means_post[:, 1], bins=30, alpha=0.7, density=True, label='Component 2')
axes[1, 1].axvline(true_means[0], color='red', linestyle='--', label='True values')
axes[1, 1].axvline(true_means[1], color='blue', linestyle='--')
axes[1, 1].set_title('Posterior Distribution of Means')
axes[1, 1].set_xlabel('Mean')
axes[1, 1].set_ylabel('Density')
axes[1, 1].legend()

# 推定分布 vs 真の分布
x_range = np.linspace(data.min(), data.max(), 1000)

# 事後平均を使った推定分布
est_weights = np.mean(weights_post, axis=0)
est_means = np.mean(means_post, axis=0)
est_variances = np.mean(variances_post, axis=0)

est_density = (est_weights[0] * stats.norm.pdf(x_range, est_means[0], np.sqrt(est_variances[0])) +
               est_weights[1] * stats.norm.pdf(x_range, est_means[1], np.sqrt(est_variances[1])))

true_density = (true_weights[0] * stats.norm.pdf(x_range, true_means[0], np.sqrt(true_variances[0])) +
                true_weights[1] * stats.norm.pdf(x_range, true_means[1], np.sqrt(true_variances[1])))

axes[1, 2].hist(data, bins=30, density=True, alpha=0.7, color='lightgray', label='Data')
axes[1, 2].plot(x_range, true_density, 'r-', linewidth=2, label='True distribution')
axes[1, 2].plot(x_range, est_density, 'b--', linewidth=2, label='Estimated distribution')
axes[1, 2].set_title('Distribution Comparison')
axes[1, 2].set_xlabel('Value')
axes[1, 2].set_ylabel('Density')
axes[1, 2].legend()

plt.tight_layout()
plt.show()

## 3.5 ブロックギブスサンプリング

変数を個別に更新する代わりに、変数のブロック（グループ）を同時に更新する手法です。

In [ ]:
def block_gibbs_bivariate_normal(mu, cov, n_samples, block_prob=0.5):
    """
    ブロックギブスサンプリング（2変量正規分布）
    
    Parameters:
    - mu, cov: 分布パラメータ
    - n_samples: サンプル数
    - block_prob: ブロック更新を行う確率
    """
    samples = np.zeros((n_samples, 2))
    
    # 条件付き分布のパラメータ（通常のギブスサンプリング用）
    mu_x, mu_y = mu[0], mu[1]
    var_x, var_y = cov[0, 0], cov[1, 1]
    cov_xy = cov[0, 1]
    rho = cov_xy / np.sqrt(var_x * var_y)
    sigma_x_given_y = np.sqrt(var_x * (1 - rho**2))
    sigma_y_given_x = np.sqrt(var_y * (1 - rho**2))
    
    # 初期値
    x, y = mu_x, mu_y
    
    block_updates = 0
    
    for i in range(n_samples):
        if np.random.rand() < block_prob:
            # ブロック更新：両変数を同時に更新
            new_sample = np.random.multivariate_normal(mu, cov)
            x, y = new_sample[0], new_sample[1]
            block_updates += 1
        else:
            # 通常のギブス更新
            # x | y
            mu_x_given_y = mu_x + rho * np.sqrt(var_x / var_y) * (y - mu_y)
            x = np.random.normal(mu_x_given_y, sigma_x_given_y)
            
            # y | x
            mu_y_given_x = mu_y + rho * np.sqrt(var_y / var_x) * (x - mu_x)
            y = np.random.normal(mu_y_given_x, sigma_y_given_x)
        
        samples[i] = [x, y]
    
    print(f"ブロック更新の割合: {block_updates / n_samples:.2%}")
    
    return samples

# 高い相関を持つ分布でテスト
high_corr_cov = np.array([[1.0, 0.95], [0.95, 1.0]])

print("高相関分布での比較...")
print(f"相関係数: {high_corr_cov[0,1]/np.sqrt(high_corr_cov[0,0]*high_corr_cov[1,1]):.3f}")

# 通常のギブス
samples_gibbs_high = gibbs_sampling_bivariate_normal(mu, high_corr_cov, 5000)

# ブロックギブス（50%の確率でブロック更新）
samples_block_gibbs = block_gibbs_bivariate_normal(mu, high_corr_cov, 5000, block_prob=0.5)

# 効率比較
gibbs_high_autocorr, gibbs_high_eff = compute_efficiency_metrics(samples_gibbs_high)
block_autocorr, block_eff = compute_efficiency_metrics(samples_block_gibbs)

print(f"\n=== 高相関での効率比較 ===")
print(f"{'Method':<15} {'Dim':<5} {'Autocorr Time':<15} {'Eff Sample Size':<18}")
print("-" * 55)
for dim in range(2):
    print(f"{'Regular Gibbs':<15} {'X'+str(dim+1):<5} {gibbs_high_autocorr[dim]:<15d} {gibbs_high_eff[dim]:<18.1f}")
    print(f"{'Block Gibbs':<15} {'X'+str(dim+1):<5} {block_autocorr[dim]:<15d} {block_eff[dim]:<18.1f}")
    print()

print(f"平均効率サンプルサイズ:")
print(f"  通常ギブス:   {np.mean(gibbs_high_eff):.1f}")
print(f"  ブロックギブス: {np.mean(block_eff):.1f}")

## 3.6 演習問題

### 問題1：3変量正規分布のギブスサンプリング
3変量正規分布に対してギブスサンプリングを実装しなさい。

In [ ]:
# 問題1の解答欄
def gibbs_sampling_trivariate_normal(mu, cov, n_samples):
    """
    3変量正規分布からのギブスサンプリング
    
    ヒント：
    3変量正規分布 N(μ, Σ) において、
    X1 | X2, X3 ~ N(μ1 + Σ12 Σ22^-1 (X23 - μ23), Σ11 - Σ12 Σ22^-1 Σ21)
    ここで X23 = [X2, X3]^T, μ23 = [μ2, μ3]^T
    """
    # ここに実装してください
    pass  # 学習者が実装

# テスト用パラメータ
mu_3d = np.array([1.0, 2.0, 3.0])
cov_3d = np.array([
    [2.0, 0.8, 0.3],
    [0.8, 1.5, 0.6],
    [0.3, 0.6, 1.0]
])

# samples_3d = gibbs_sampling_trivariate_normal(mu_3d, cov_3d, 5000)

### 問題2：線形回帰のベイズ推定
ギブスサンプリングを用いて線形回帰のベイズ推定を実装しなさい。

In [ ]:
# 問題2の解答欄
def gibbs_bayesian_regression(X, y, n_iterations, 
                             prior_beta_var=10.0, 
                             prior_sigma_shape=1.0, prior_sigma_rate=1.0):
    """
    線形回帰のベイズ推定
    
    モデル: y = X β + ε, ε ~ N(0, σ²)
    事前分布: β ~ N(0, prior_beta_var * I), σ² ~ InvGamma(shape, rate)
    
    ヒント：
    - β | σ², y ~ N(posterior_mean, posterior_cov)
    - σ² | β, y ~ InvGamma(posterior_shape, posterior_rate)
    """
    # ここに実装してください
    pass  # 学習者が実装

# テストデータ生成
np.random.seed(42)
n_obs = 100
X_reg = np.column_stack([np.ones(n_obs), np.random.randn(n_obs, 2)])
true_beta = np.array([1.0, 2.0, -1.5])
y_reg = X_reg @ true_beta + np.random.normal(0, 0.8, n_obs)

print(f"真の回帰係数: {true_beta}")
print(f"真の誤差分散: 0.64")

# results_reg = gibbs_bayesian_regression(X_reg, y_reg, 2000)

## まとめ

この章では、ギブスサンプリングについて以下を学習しました：

1. **基本原理**：条件付き分布からの順次サンプリング
2. **利点**：
   - 受理率100%
   - 自動的な詳細釣り合い
   - 計算効率の良さ
3. **実装例**：
   - 多変量正規分布
   - 混合モデルのパラメータ推定
4. **応用技術**：
   - ブロックギブスサンプリング
   - 潜在変数モデル

### 重要なポイント
- **条件付き分布の導出**：解析的に求められることが前提
- **高相関時の問題**：変数間の相関が高いと収束が遅い
- **ブロック更新**：相関問題の解決策の一つ
- **潜在変数**：観測されない変数も自然に扱える

### メトロポリス・ヘイスティングス法との使い分け
- **ギブス**：条件付き分布が既知で簡単にサンプリングできる場合
- **MH**：条件付き分布が複雑で直接サンプリングが困難な場合
- **混合**：部分的にギブス、部分的にMHを使うハイブリッド手法も有効

次の章では、これらのMCMC手法の収束を診断し、性能を評価する方法を学習します。